In [196]:
from typing import Dict, List, Any, Hashable, Tuple
from datetime import datetime
import pandas as pd
import hashlib
import json

In [197]:
SINGLE_FIELDS = {
    # Identification
    "nct_id": "protocolSection.identificationModule.nctId",
    "brief_title": "protocolSection.identificationModule.briefTitle",
    "official_title": "protocolSection.identificationModule.officialTitle",
    "acronym": "protocolSection.identificationModule.acronym",
    "org_study_id": "protocolSection.identificationModule.orgStudyIdInfo.id",
    # Description
    "brief_summary": "protocolSection.descriptionModule.briefSummary",
    "detailed_desc": "protocolSection.descriptionModule.detailedDescription",

    # Sponsor
    "responsible_party": "protocolSection.sponsorCollaboratorsModule.responsibleParty.type",
    
    # Design (single values)
    "study_type": "protocolSection.designModule.studyType",
    "patient_registry": "protocolSection.designModule.patientRegistry",
    "enrollment_type": "protocolSection.designModule.enrollmentInfo.type",
    "enrollment_count": "protocolSection.designModule.enrollmentInfo.count",
    "design_allocation": "protocolSection.designModule.designInfo.allocation",
    "design_intervention_model": "protocolSection.designModule.designInfo.interventionModel",
    "design_intervention_model_desc": "protocolSection.designModule.designInfo.interventionModelDescription",
    "design_primary_purpose": "protocolSection.designModule.designInfo.primaryPurpose",
    "design_observational_model": "protocolSection.designModule.designInfo.observationalModel",
    "design_time_perspective": "protocolSection.designModule.designInfo.timePerspective",
    "design_masking": "protocolSection.designModule.designInfo.maskingInfo.masking",
    # Biospecimen
    "biospec_retention": "protocolSection.designModule.bioSpec.retention",
    "biospec_desc": "protocolSection.designModule.bioSpec.description",
    # Eligibility
    "eligibility_criteria": "protocolSection.eligibilityModule.eligibilityCriteria",
    "healthy_volunteers": "protocolSection.eligibilityModule.healthyVolunteers",
    "sex": "protocolSection.eligibilityModule.sex",
    "min_age": "protocolSection.eligibilityModule.minimumAge",
    "max_age": "protocolSection.eligibilityModule.maximumAge",
    "population_desc": "protocolSection.eligibilityModule.studyPopulation",
    "sampling_method": "protocolSection.eligibilityModule.samplingMethod",
    # Status
    "overall_status": "protocolSection.statusModule.overallStatus",
    "last_known_status": "protocolSection.statusModule.lastKnownStatus",
    "status_verified_date": "protocolSection.statusModule.statusVerifiedDate",
    "start_date": "protocolSection.statusModule.startDateStruct.date",
    "start_date_type": "protocolSection.statusModule.startDateStruct.type",
    "first_submit_date": "protocolSection.statusModule.studyFirstSubmitDate",
    "last_update_submit_date": "protocolSection.statusModule.lastUpdateSubmitDate",
    "completion_date": "protocolSection.statusModule.completionDateStruct.date",
    "completion_date_type": "protocolSection.statusModule.completionDateStruct.type",
    "why_stopped": "protocolSection.statusModule.whyStopped",
    "has_expanded_access": "protocolSection.statusModule.expandedAccessInfo.hasExpandedAccess",
    # Oversight
    "has_dmc": "protocolSection.oversightModule.oversightHasDmc",
    "is_fda_regulated_drug": "protocolSection.oversightModule.isFdaRegulatedDrug",
    "is_fda_regulated_device": "protocolSection.oversightModule.isFdaRegulatedDevice",
    "is_unapproved_device": "protocolSection.oversightModule.isUnapprovedDevice",
    "is_us_export": "protocolSection.oversightModule.isUsExport",
    # Individual participant data
    "ipd_sharing": "protocolSection.ipdSharingStatementModule.ipdSharing",
    "ipd_desc": "protocolSection.ipdSharingStatementModule.description",
    "ipd_time_frame": "protocolSection.ipdSharingStatementModule.timeFrame",
    "ipd_access_criteria": "protocolSection.ipdSharingStatementModule.accessCriteria",
    "ipd_url": "protocolSection.ipdSharingStatementModule.url",
    # Miscellaneous
    "version_holder": "derivedSection.miscInfoModule.versionHolder",
    "has_results": "hasResults",
    "last_updated": "protocolSection.statusModule.lastUpdatePostDateStruct.date",
    "limitations_desc": "resultsSection.moreInfoModule.limitationsAndCaveats.description",
    # Certain agreements
    "certain_agreement_pi_sponsor_employee": "resultsSection.moreInfoModule.certainAgreement.piSponsorEmployee",
    "certain_agreement_restrictive": "resultsSection.moreInfoModule.certainAgreement.restrictiveAgreement",
    "certain_agreement_other_details": "resultsSection.moreInfoModule.certainAgreement.otherDetails",
    "certain_agreement_restriction_type": "resultsSection.moreInfoModule.certainAgreement.restrictionType",
    # Point of contact
    "poc_title": "resultsSection.moreInfoModule.pointOfContact.title",
    "poc_organization": "resultsSection.moreInfoModule.pointOfContact.organization",
    "poc_email": "resultsSection.moreInfoModule.pointOfContact.email",
    "poc_phone": "resultsSection.moreInfoModule.pointOfContact.phone",
    "poc_phone_ext": "resultsSection.moreInfoModule.pointOfContact.phoneExt",
    # Submission tracking
    "sub_tracking_estimated_results_date": "derivedSection.miscInfoModule.submissionTracking.estimatedResultsFirstSubmitDate",
}

NESTED_FIELDS = {
    "sponsor": {
        "index_field": "protocolSection.sponsorCollaboratorsModule.leadSponsor",
        "object_type": "simple dict",
        "fields": [
            ("lead_sponsor_name", "name"),
            ("lead_sponsor_class", "class"),
        ],
        "table_name": "sponsors",
        "bridge_table_name": "study_sponsors",
        "transformer_method": "extract_sponsors",
    },
    "collaborators": {
        "index_field": "protocolSection.sponsorCollaboratorsModule.collaborators",
        "object_type": "array_of_dicts",
        "fields": [
            ("sponsor_name", "name"),
            ("sponsor_class", "class"),
        ],
        "table_name": "sponsor",
        "bridge_table_name": "study_sponsors",
        "transformer_method": "extract_sponsors",
    },
    "conditions": {
        "index_field": "protocolSection.conditionsModule.conditions",
        "object_type": "simple_array",
        "table_name": "conditions",
        "bridge_table_name": "bridge_study_conditions",
        "field_name": "condition_name",
        "transformer_method": "extract_conditions",
    },
    "keywords": {
        "index_field": "protocolSection.conditionsModule.keywords",
        "object_type": "simple_array",
        "table_name": "keywords",
        "bridge_table_name": "bridge_study_keywords",
        "field_name": "keyword",
        "transformer_method": "extract_keywords",
    },
    "arm_groups": {
        "index_field": "protocolSection.armsInterventionsModule.armGroups",
        "object_type": "array_of_dicts",
        "table_name": "study_arm_groups",
        "fields": [
            ("arm_group_label", "label"),
            ("arm_group_type", "type"),
            ("arm_group_desc", "description"),
        ],
        "nested": {
            "interventionNames": {
                "object_type": "nested_simple_array",
                "bridge_table_name": "arm_group_interventions",
                "field_name": "intervention_name",
            },
            "transformer_method": "extract_arm_groups",
        },
    },
    "interventions": {
        "index_field": "protocolSection.armsInterventionsModule.interventions",
        "object_type": "array_of_dicts",
        "fields": [
            ("intervention_name", "name"),
            ("intervention_desc", "description"),
            ("intervention_type", "type"),
        ],
        "nested": {
            "otherNames": {
                "object_type": "nested_simple_array",
                "table_name": "interventions",
                "bridge_table_name": "bridge_table_name",
            }
        },
        "table_name": "interventions",
        "bridge_table_name": "bridge_study_interventions",
        "transformer_method": "extract_interventions",
    },

}

In [198]:
def generate_key(*args) -> str:
    """Generates a deterministic surrogate key from input values."""
    combined = "|".join(str(arg) for arg in args if arg is not None)
    return hashlib.sha256(combined.encode()).hexdigest()[:16]

In [199]:
def extract_study_fields(study_key: str, study_data: pd.Series) -> Dict:
    study_record = dict()

    study_record['study_key'] = study_key
    for entity_key in SINGLE_FIELDS:
        index_field = SINGLE_FIELDS.get(entity_key)

        study_record[entity_key] = study_data.get(index_field)

    return study_record

In [211]:
def extract_sponsors(idx: int, study_key: str, study_data: pd.Series):

    sponsors = []
    study_sponsors = []

    # Extract lead sponsor
    lead_sponsor_index = NESTED_FIELDS["sponsor"]["index_field"]

    # sponsor name and class are scalar values and MUST be extracted directly
    lead_sponsor_name = study_data.get(f'{lead_sponsor_index}.name')
    lead_sponsor_class = study_data.get(f'{lead_sponsor_index}.class')

    if pd.notna(lead_sponsor_name) and pd.notna(lead_sponsor_class):
        sponsor_key = generate_key(
            lead_sponsor_name, lead_sponsor_class
        )
        sponsors.append(
            {
                "sponsor_key": sponsor_key,
                "name": lead_sponsor_name,
                "sponsor_class": lead_sponsor_class,
            }
        )

        study_sponsors.append(
            {"study_key": study_key, "sponsor_key": sponsor_key, "is_lead": True}
        )
    else:
        print(f"No lead sponsor found for {idx}")

    # Extract collaborators
    collaborators_index = NESTED_FIELDS["collaborators"]["index_field"]
    collaborators_list = study_data.get(collaborators_index)

    if collaborators_list is not None and len(collaborators_list) > 0:
        for collaborator in collaborators_list:
            sponsor_key = generate_key(
                collaborator.get("name"), collaborator.get("class")
            )

            sponsors.append(
                {
                    "sponsor_key": sponsor_key,
                    "name": collaborator.get("name"),
                    "sponsor_class": collaborator.get("class"),
                }
            )

            study_sponsors.append(
                {"study_key": study_key, "sponsor_key": sponsor_key, "is_lead": False}
            )

    return sponsors, study_sponsors

In [212]:
def extract_conditions(idx: int, study_key: str, study_data: pd.Series) -> Tuple | None:
    conditions = []
    study_conditions = []

    conditions_index = NESTED_FIELDS["conditions"]["index_field"]
    conditions_list = study_data.get(conditions_index)


    if conditions_list is not None and len(conditions_list) > 0:
        for condition in conditions_list:
            condition_key = generate_key(condition)

            conditions.append(
                {"condition_key": condition_key, "condition_name": condition}
            )

            study_conditions.append(
                {
                    "study_key": study_key,
                    "condition_key": condition_key,
                }
            )

        return conditions, study_conditions

    print(f"No conditions found for {idx}")
    return conditions, study_conditions

In [223]:
def extract_keywords(idx: Hashable, study_key: str, study_data: pd.Series) -> Tuple:
    keywords = []
    study_keywords = []

    keywords_index = NESTED_FIELDS["keywords"]["index_field"]
    keywords_list = study_data.get(keywords_index)
    if keywords_list is not None and len(keywords_list) > 0:
        for keyword in keywords_list:
            keyword_key = generate_key(keyword)

            keywords.append({"keyword_key": keyword_key, "keyword_name": keyword})

            study_keywords.append(
                {
                    "study_key": study_key,
                    "keyword_key": keyword_key,
                }
            )
        
    return keywords, study_keywords

In [229]:
all_studies = []
all_sponsors = []
all_study_sponsors = []

all_conditions = []
all_study_conditions = []

all_keywords = []
all_study_keywords = []

all_study_arm_groups = []
all_study_arm_group_interventions = []

all_interventions = []
all_interventions_other_names = []
all_study_interventions = []

df = pd.read_parquet("1.parquet")
df_studies = pd.json_normalize(df['studies'].tolist())

for idx, study in df_studies.iterrows():
    nct_index = SINGLE_FIELDS['nct_id']
    nct_id = study.get(nct_index)
    
    study_key = generate_key(nct_id)


    #study
    study_record = extract_study_fields(study_key, study)
    all_studies.append(study_record)

    # sponsors
    sponsors, study_sponsors = extract_sponsors(idx, study_key, study)
    all_sponsors.extend(sponsors)
    all_study_sponsors.extend(study_sponsors)

    # conditions and keywords
    conditions, study_conditions = extract_conditions(idx, study_key, study)
    all_conditions.extend(conditions)
    all_study_conditions.extend(study_conditions)

    keywords, study_keywords = extract_keywords(idx, study_key, study)
    all_keywords.extend(keywords)
    all_study_keywords.extend(study_keywords)


studies = pd.DataFrame(all_studies)

df_sponsors = pd.DataFrame(all_sponsors)
df_study_sponsors = pd.DataFrame(all_study_sponsors)

df_conditions = pd.DataFrame(all_conditions)
df_study_conditions= pd.DataFrame(all_study_conditions)

df_keywords = pd.DataFrame(all_keywords)
df_study_keywords = pd.DataFrame(all_study_keywords)


print(f"STUDIES {len(studies)}")

#dedupe
print(f"SPONSORS {len(df_sponsors)}")
print(f"STUDY SPONSORS {len(df_study_sponsors)}")
df_sponsors = df_sponsors.drop_duplicates(subset=["sponsor_key"])
print(f"DEDUPED SPONSORS {len(df_sponsors)}")

print(f"CONDITIONS {len(df_conditions)}")
print(f"STUDY CONDITIONS {len(df_study_conditions)}")
df_conditions = df_conditions.drop_duplicates(subset=["condition_key"])
print(f"DEDUPED CONDITIONS {len(df_conditions)}")

print(f"KEYWORDS {len(df_keywords)}")
print(f"STUDY KEYWORDS {len(df_study_keywords)}")
df_keywords = df_keywords.drop_duplicates(subset=["keyword_key"])
print(f"DEDUPED KEYWORDS {len(df_keywords)}")


STUDIES 1000
SPONSORS 1631
STUDY SPONSORS 1631
DEDUPED SPONSORS 1176
CONDITIONS 1754
STUDY CONDITIONS 1754
DEDUPED CONDITIONS 1355
KEYWORDS 2419
STUDY KEYWORDS 2419
DEDUPED KEYWORDS 2235


In [230]:
studies.to_csv("data/study_data.csv", index=False)

In [231]:
df_sponsors.to_csv("data/sponsors.csv", index=False)

In [232]:
df_study_sponsors.to_csv("data/bridge_study_sponsors.csv", index=False)

In [233]:
df_conditions.to_csv("data/conditions.csv", index=False)

In [234]:
df_study_conditions.to_csv("data/study_conditions.csv", index=False)

In [235]:
df_keywords.to_csv("data/keywords.csv", index=False)

In [236]:
df_study_keywords.to_csv("data/study_keywords.csv", index=False)